<a href="https://colab.research.google.com/github/jacobazevedojr/CECS-451-Assignment-3/blob/main/genetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Genetic Algorithms (Local Search)

# Implement a program that performs the Genetic algorithm with 8
# states including the three operations, i.e., selection, crossover, mutation 
# to find a solution.

from board import Board
import numpy as np
import random

ModuleNotFoundError: ignored

In [2]:
# Driver

def start(k, n):
  # Initialize k random states of n X n boards
  states = []
  for i in range(k):
    states.append(Board(n))

  noSolution = True
  while noSolution:
    # Initializes fitness values for all states
    parents = selection(states)
    newStates = crossover(parents)
    states = mutation(newStates)

    # bestFitness is the best fitness for all states (***INIT THESE VALUES***)
    bestFitness = 0
    optimalFitness = 0
    if (bestFitness == optimalFitness):
      noSolution = False

In [4]:
# The more fit a state is, the higher likelihood that it is chosen to mate
# There is a likelihood of high fitness states "breeding" with more than one 
# "partners" and low fitness states not breeding at all

# Pass the current k states and return the parents

def selection(states):
  for state in states:
    # Initialize fitness for each state
    state.fitness()

  # We need a way to translate fitness into a percentage
  # With non-attacking queens, bigger is better, but if we try to look at 
  # attacking queens, smaller is better

  cumulativeWeights = []
  maxTotal = len(states) * maxNonAttackingPairs
  cumulativeWeights.append((maxAttackingPairs - states[0].fit)/maxAttackingPairs)
  # Hopefully this works
  for i, state in enumerate(states[1:]):
    # Builds the cumulative weight array 
    # Each array adds the curr percent with the last to determine percent range
    cumulativeWeights.append(states[i - 1] + (maxAttackingPairs - state.fit)/maxAttackingPairs)

  parents = []
  # Use random number generator to choose who "mates"
  for i in range(states[0].n_queen):
    rand = random.random()
    for i, weight in enumerate(cumulativeWeights):
      # ex: | .32 - .56 - .70 - .89 - 1 |
      # The first time the rand is less than a number, it will fall within that
      # index range
      if random < weight:
        # The parent is chosen
        parents.append(states[i])
        # *** URGENT *** CURRENTLY STATES CAN MATE WITH THEMSELVES NEEDS FIX
        break

  return parents

In [5]:
# Once the selection of mates has been decided, they must now... do the deed

# Pass the selected parent states (always even) 
# and "breed" them, return their children
def crossover(states):
  children = []
  for i in range(len(states)//2):
    parent1 = states[i]
    parent2 = states[i + 1]
    # Every pair of parents (beginning with 0 and 1) mate with each other
    # Each pair of parents make a pair of children (twins)
    tempTwins = mate(parent1, parent2)
    for child in tempTwins:
      children.append(child)

  return children

In [6]:
# A random position within the encoded state representation is "mutated"
# The mutated value will be a random value between 0 and (n_queen - 1)

def mutation(states):
  n = states[0].n_queen
  for state in states:
    randIndex = np.random.randint(0, n)
    # state.encoding[randIndex] = np.random.randint(0, n)
  return states

In [7]:
# Returns the child for two parents
def mate(parent1, parent2):
  # Determine some arbitrary slicing point
  n = parent1.n_queen
  sliceIndex = np.random.randint(0, n)

  twins = []
  twins.append(parent1[0:n] + parent2[n:])
  twins.append(parent2[0:n] + parent1[n:])

  return twins

In [9]:
start(8, 5)

NameError: ignored